## Artificial and Computational Intelligence Assignment 2

## Gaming with Min-Max Algorithm - Solution template

### List only the BITS (Name) of active contributors in this assignment:
1. ___________________
2. __________________
3. ____________________
4. ___________________
5. ___________________

# Things to follow

1. Use appropriate data structures to represent the graph using python libraries
2. Provide proper documentation
3. Create neat solution without error during game playing

### Coding begins here

### PEAS - Data structures and fringes that define the Agent environment goes here

## Applying the PEAS Model to the Catch-Up with Numbers Game

To apply the PEAS (Performance measure, Environment, Actuators, Sensors) model to the Catch-Up with Numbers game, we first need to define each component of the PEAS model and then explore the specific data structures and fringes used to implement the agent and its interactions within the game environment.

### PEAS Model for Catch-Up with Numbers Game

#### Performance Measure (P):
- **Goal**: Maximize the numeric sum while adhering to the game's rules.
- **Efficiency**: Minimize the number of turns taken to reach a conclusion.
- **Fair Play**: Adhere strictly to the turn-taking and number-choosing rules outlined in the game.

#### Environment (E):
- **Type**: Discrete, deterministic, and fully observable.
- **Composition**: The set of available numbers: \(1, 2, 3, ... n\).
- **Dynamics**: Static during a player’s turn (the environment doesn’t change unless a player acts).

#### Actuators (A):
- **Choosing Numbers**: The agent's actions include selecting one or more numbers from the set, which influences the state of the game environment.

#### Sensors (S):
- **Number Availability**: The agent senses or perceives which numbers are still available to be chosen after each turn.


### Implementation of the Min-Max algorithm

### Implementation of the alpha-beta pruning  

### Choice and implementation of the Static Evaluation Function.

### Interactive Implementation and Dynamic Inputs

### Performance Comparison of the Two Algorithms


### Min Max - (Plain)

In [1]:
import random
import itertools

# Function to determine the winner of the game
def determine_winner(p1_total, p2_total):
    if p1_total > p2_total:
        return 1  # P1 wins
    elif p2_total > p1_total:
        return -1  # P2 wins
    else:
        return 0  # It's a tie

# Function to select numbers incrementally until total equals or exceeds opponent's total
def incremental_selection(remaining_numbers, player_total, target_total):
    selected_numbers = []
    current_total = player_total

    # Keep selecting numbers until the player's total equals or exceeds the opponent's total
    while current_total < target_total and remaining_numbers:
        # Randomly select one number from the remaining numbers
        random_choice = random.choice(remaining_numbers)
        selected_numbers.append(random_choice)
        current_total += random_choice
        remaining_numbers.remove(random_choice)

        # Stop once the current total meets or exceeds the opponent's total
        if current_total >= target_total:
            break

    return current_total, selected_numbers, remaining_numbers

# Expectiminimax algorithm with incremental selection
def expectiminimax(remaining_numbers, p1_total, p2_total, is_p1_turn):
    if not remaining_numbers:  # Base case: no numbers left, check the winner
        return determine_winner(p1_total, p2_total), []

    if is_p1_turn:
        max_eval = float('-inf')
        best_move = []
        for subset in get_valid_subsets(remaining_numbers, p2_total, p1_total):
            new_remaining_numbers = [x for x in remaining_numbers if x not in subset]
            new_p1_total = p1_total + sum(subset)
            eval, _ = expectiminimax(new_remaining_numbers, new_p1_total, p2_total, False)
            if eval > max_eval:
                max_eval = eval
                best_move = subset
        return max_eval, best_move
    else:
        if p2_total >= p1_total:  # If P2 can match or exceed P1's total, it plays optimally
            min_eval = float('inf')
            best_move = []
            for subset in get_valid_subsets(remaining_numbers, p1_total, p2_total):
                new_remaining_numbers = [x for x in remaining_numbers if x not in subset]
                new_p2_total = p2_total + sum(subset)
                eval, _ = expectiminimax(new_remaining_numbers, p1_total, new_p2_total, True)
                if eval < min_eval:
                    min_eval = eval
                    best_move = subset
            return min_eval, best_move
        else:
            # If P2 cannot match or exceed P1's total, find the closest subset
            closest_subset = get_closest_subset(remaining_numbers, p1_total, p2_total)
            return -1, closest_subset  # P2 tries to reduce the loss margin

# Function to get valid subsets (combining numbers to exceed the opponent's total)
def get_valid_subsets(remaining_numbers, target_total, current_total):
    valid_subsets = []
    for r in range(1, len(remaining_numbers) + 1):
        for subset in itertools.combinations(remaining_numbers, r):
            if current_total + sum(subset) >= target_total:
                valid_subsets.append(subset)
    return valid_subsets

# Function to get the closest subset if P2 cannot exceed P1
def get_closest_subset(remaining_numbers, target_total, current_total):
    best_diff = float('inf')
    best_subset = []
    
    for r in range(1, len(remaining_numbers) + 1):
        for subset in itertools.combinations(remaining_numbers, r):
            subset_sum = sum(subset)
            new_total = current_total + subset_sum
            if new_total >= target_total:  # Skip if it exceeds P1's score
                continue
            diff = target_total - new_total
            if diff < best_diff:  # Find the closest
                best_diff = diff
                best_subset = subset
    
    return best_subset

# Function to handle random turn based on game rules
def expect_random_turn(remaining_numbers, player_total, opponent_total):
    valid_subsets = get_valid_subsets(remaining_numbers, opponent_total, player_total)
    if not valid_subsets:
        return player_total, [], remaining_numbers

    # Randomly pick a valid subset that satisfies the condition
    random_choice = random.choice(valid_subsets)
    current_total = player_total + sum(random_choice)
    remaining_numbers = [x for x in remaining_numbers if x not in random_choice]
    
    return current_total, random_choice, remaining_numbers

# Main function to run the game using Expectiminimax and incremental selection
def run_expectiminimax_game(n):
    remaining_numbers = list(range(1, n + 1))  # Initialize remaining numbers
    p1_total = 0  # Player 1 total score
    p2_total = 0  # Player 2 total score
    is_p1_turn = True  # Flag to switch between players
    first_turn = True  # Flag for the first move
    
    while remaining_numbers:
        if is_p1_turn:
            print("\nP1's turn")
            if first_turn:  # First move, P1 selects only one number
                first_choice = random.choice(remaining_numbers)  # Randomly select the first number
                p1_total += first_choice
                remaining_numbers.remove(first_choice)
                print(f"P1 chooses: [{first_choice}], P1 total: {p1_total}")
                first_turn = False
            else:
                # P1 selects numbers incrementally until total >= P2's total
                p1_total, selected_numbers, remaining_numbers = incremental_selection(
                    remaining_numbers, p1_total, p2_total
                )
                if not selected_numbers:  # If no more valid moves, end the game
                    print("No valid moves for P1. Game ends.")
                    break
                print(f"P1 chooses: {selected_numbers}, P1 total: {p1_total}")
                if p2_total + sum(remaining_numbers) < p1_total:  # If P2 cannot exceed P1
                    print("P1 wins the game, no need for further selection")
                    break
        else:
            print("\nP2's turn")
            # P2 selects numbers incrementally until total >= P1's total
            p2_total, selected_numbers, remaining_numbers = incremental_selection(
                remaining_numbers, p2_total, p1_total
            )
            if not selected_numbers:  # If no more valid moves, end the game
                print("No valid moves for P2. Game ends.")
                break
            print(f"P2 chooses: {selected_numbers}, P2 total: {p2_total}")
        
        is_p1_turn = not is_p1_turn  # Switch player turns

        # Break if no more numbers are available to pick
        if not remaining_numbers:
            print("No more numbers remaining. Game ends.")
            break
    
    # Determine and print the winner
    winner = determine_winner(p1_total, p2_total)
    if winner == 1:
        print("\nP1 wins with a total of", p1_total)
    elif winner == -1:
        print("\nP2 wins with a total of", p2_total)
    else:
        print("\nThe game is a tie!")

# Running the game with numbers from 1 to 20
run_expectiminimax_game(10)



P1's turn
P1 chooses: [6], P1 total: 6

P2's turn
P2 chooses: [10], P2 total: 10

P1's turn
P1 chooses: [4], P1 total: 10

P2's turn
No valid moves for P2. Game ends.

The game is a tie!


### Min Max with - Improved Solution

In [4]:
import random
import itertools

# Function to determine the winner of the game
def determine_winner(p1_total, p2_total):
    if p1_total > p2_total:
        return 1  # P1 wins
    elif p2_total > p1_total:
        return -1  # P2 wins
    else:
        return 0  # It's a tie

# Function to get valid subsets (combining numbers to exceed the opponent's total)
def get_valid_subsets(remaining_numbers, target_total, current_total):
    valid_subsets = []
    for r in range(1, len(remaining_numbers) + 1):
        for subset in itertools.combinations(remaining_numbers, r):
            if current_total + sum(subset) >= target_total:
                valid_subsets.append(subset)
    return valid_subsets

# Function to select numbers incrementally until total equals or exceeds opponent's total
def incremental_selection(remaining_numbers, player_total, target_total):
    selected_numbers = []
    current_total = player_total

    # Keep selecting numbers until the player's total equals or exceeds the opponent's total
    while current_total < target_total and remaining_numbers:
        random_choice = random.choice(remaining_numbers)
        selected_numbers.append(random_choice)
        current_total += random_choice
        remaining_numbers.remove(random_choice)

        if current_total >= target_total:
            break

    return current_total, selected_numbers, remaining_numbers

# Function to get the closest subset if P2 cannot exceed P1
def get_closest_subset(remaining_numbers, target_total, current_total):
    best_diff = float('inf')
    best_subset = []
    
    for r in range(1, len(remaining_numbers) + 1):
        for subset in itertools.combinations(remaining_numbers, r):
            subset_sum = sum(subset)
            new_total = current_total + subset_sum
            if new_total >= target_total:  # Skip if it exceeds P1's score
                continue
            diff = target_total - new_total
            if diff < best_diff:  # Find the closest
                best_diff = diff
                best_subset = subset
    
    return best_subset

# Expectiminimax algorithm with incremental selection
def expectiminimax(remaining_numbers, p1_total, p2_total, is_p1_turn):
    if not remaining_numbers:  # Base case: no numbers left, check the winner
        return determine_winner(p1_total, p2_total), []

    if is_p1_turn:
        max_eval = float('-inf')
        best_move = []
        for subset in get_valid_subsets(remaining_numbers, p2_total, p1_total):
            new_remaining_numbers = [x for x in remaining_numbers if x not in subset]
            new_p1_total = p1_total + sum(subset)
            eval, _ = expectiminimax(new_remaining_numbers, new_p1_total, p2_total, False)
            if eval > max_eval:
                max_eval = eval
                best_move = subset
        return max_eval, best_move
    else:
        if p2_total >= p1_total:  # If P2 can match or exceed P1's total, it plays optimally
            min_eval = float('inf')
            best_move = []
            for subset in get_valid_subsets(remaining_numbers, p1_total, p2_total):
                new_remaining_numbers = [x for x in remaining_numbers if x not in subset]
                new_p2_total = p2_total + sum(subset)
                eval, _ = expectiminimax(new_remaining_numbers, p1_total, new_p2_total, True)
                if eval < min_eval:
                    min_eval = eval
                    best_move = subset
            return min_eval, best_move
        else:
            # If P2 cannot match or exceed P1's total, find the closest subset
            closest_subset = get_closest_subset(remaining_numbers, p1_total, p2_total)
            return -1, closest_subset  # P2 tries to reduce the loss margin

# Main function to run the game using Expectiminimax and incremental selection
def run_expectiminimax_game(n):
    remaining_numbers = list(range(1, n + 1))  # Initialize remaining numbers
    p1_total = 0  # Player 1 total score
    p2_total = 0  # Player 2 total score
    is_p1_turn = True  # Flag to switch between players
    first_turn = True  # Flag for the first move
    
    while remaining_numbers:
        if is_p1_turn:
            print("\nP1's turn")
            if first_turn:  # First move, P1 selects only one number
                first_choice = random.choice(remaining_numbers)  # Randomly select the first number
                p1_total += first_choice
                remaining_numbers.remove(first_choice)
                print(f"P1 chooses: [{first_choice}], P1 total: {p1_total}")
                first_turn = False
            else:
                # P1 selects numbers incrementally until total >= P2's total
                p1_total, selected_numbers, remaining_numbers = incremental_selection(
                    remaining_numbers, p1_total, p2_total
                )
                if not selected_numbers:  # If no more valid moves, end the game
                    print("No valid moves for P1. Game ends.")
                    break
                print(f"P1 chooses: {selected_numbers}, P1 total: {p1_total}")
                if p2_total + sum(remaining_numbers) < p1_total:  # If P2 cannot exceed P1
                    print("P1 wins the game, no need for further selection")
                    break
        else:
            print("\nP2's turn")
            # P2 selects numbers incrementally until total >= P1's total
            p2_total, selected_numbers, remaining_numbers = incremental_selection(
                remaining_numbers, p2_total, p1_total
            )
            if not selected_numbers:  # If no more valid moves, end the game
                print("No valid moves for P2. Game ends.")
                break
            print(f"P2 chooses: {selected_numbers}, P2 total: {p2_total}")
        
        is_p1_turn = not is_p1_turn  # Switch player turns

        # Break if no more numbers are available to pick
        if not remaining_numbers:
            print("No more numbers remaining. Game ends.")
            break
    
    # Determine and print the winner
    winner = determine_winner(p1_total, p2_total)
    if winner == 1:
        print("\nP1 wins with a total of", p1_total)
    elif winner == -1:
        print("\nP2 wins with a total of", p2_total)
    else:
        print("\nThe game is a tie!")

# Running the game with numbers from 1 to 20
run_expectiminimax_game(10)



P1's turn
P1 chooses: [2], P1 total: 2

P2's turn
P2 chooses: [5], P2 total: 5

P1's turn
P1 chooses: [10], P1 total: 12

P2's turn
P2 chooses: [4, 9], P2 total: 18

P1's turn
P1 chooses: [7], P1 total: 19

P2's turn
P2 chooses: [1], P2 total: 19

P1's turn
No valid moves for P1. Game ends.

The game is a tie!


### Min Max With Alpha Beta Pruning

In [5]:
import random
import itertools

# Function to determine the winner of the game
def determine_winner(p1_total, p2_total):
    if p1_total > p2_total:
        return 1  # P1 wins
    elif p2_total > p1_total:
        return -1  # P2 wins
    else:
        return 0  # It's a tie

# Function to get valid subsets (combining numbers to exceed the opponent's total)
def get_valid_subsets(remaining_numbers, target_total, current_total):
    valid_subsets = []
    for r in range(1, len(remaining_numbers) + 1):
        for subset in itertools.combinations(remaining_numbers, r):
            if current_total + sum(subset) >= target_total:
                valid_subsets.append(subset)
    return valid_subsets

# Function to select numbers incrementally until total equals or exceeds opponent's total
def incremental_selection(remaining_numbers, player_total, target_total):
    selected_numbers = []
    current_total = player_total

    # Keep selecting numbers until the player's total equals or exceeds the opponent's total
    while current_total < target_total and remaining_numbers:
        random_choice = random.choice(remaining_numbers)
        selected_numbers.append(random_choice)
        current_total += random_choice
        remaining_numbers.remove(random_choice)

        if current_total >= target_total:  # Stop as soon as we exceed or match the target
            break

    return current_total, selected_numbers, remaining_numbers

# Expectiminimax algorithm with Alpha-Beta pruning
def expectiminimax_alpha_beta(remaining_numbers, p1_total, p2_total, is_p1_turn, alpha, beta):
    if not remaining_numbers:  # Base case: no numbers left, check the winner
        return determine_winner(p1_total, p2_total), []

    if is_p1_turn:
        max_eval = float('-inf')
        best_move = []
        for subset in get_valid_subsets(remaining_numbers, p2_total, p1_total):
            new_remaining_numbers = [x for x in remaining_numbers if x not in subset]
            new_p1_total = p1_total + sum(subset)
            eval, _ = expectiminimax_alpha_beta(new_remaining_numbers, new_p1_total, p2_total, False, alpha, beta)
            if eval > max_eval:
                max_eval = eval
                best_move = subset
            alpha = max(alpha, eval)  # Update alpha
            if beta <= alpha:  # Prune branches
                break
        return max_eval, best_move
    else:
        min_eval = float('inf')
        best_move = []
        for subset in get_valid_subsets(remaining_numbers, p1_total, p2_total):
            new_remaining_numbers = [x for x in remaining_numbers if x not in subset]
            new_p2_total = p2_total + sum(subset)
            eval, _ = expectiminimax_alpha_beta(new_remaining_numbers, p1_total, new_p2_total, True, alpha, beta)
            if eval < min_eval:
                min_eval = eval
                best_move = subset
            beta = min(beta, eval)  # Update beta
            if beta <= alpha:  # Prune branches
                break
        return min_eval, best_move

# Main function to run the game using Expectiminimax with Alpha-Beta Pruning
def run_expectiminimax_alpha_beta_game(n):
    remaining_numbers = list(range(1, n + 1))  # Initialize remaining numbers
    p1_total = 0  # Player 1 total score
    p2_total = 0  # Player 2 total score
    is_p1_turn = True  # Flag to switch between players
    first_turn = True  # Flag for the first move
    alpha = float('-inf')  # Initialize alpha for P1 (maximizing)
    beta = float('inf')  # Initialize beta for P2 (minimizing)
    
    while remaining_numbers:
        if is_p1_turn:
            print("\nP1's turn")
            if first_turn:  # First move, P1 selects only one number
                first_choice = random.choice(remaining_numbers)  # Randomly select the first number
                p1_total += first_choice
                remaining_numbers.remove(first_choice)
                print(f"P1 chooses: [{first_choice}], P1 total: {p1_total}")
                first_turn = False
            else:
                # P1 selects numbers incrementally until total >= P2's total
                p1_total, selected_numbers, remaining_numbers = incremental_selection(
                    remaining_numbers, p1_total, p2_total
                )
                if not selected_numbers:  # If no more valid moves, end the game
                    print("No valid moves for P1. Game ends.")
                    break
                print(f"P1 chooses: {selected_numbers}, P1 total: {p1_total}")
                if p2_total + sum(remaining_numbers) < p1_total:  # If P2 cannot exceed P1
                    print("P1 wins the game, no need for further selection")
                    break
        else:
            print("\nP2's turn")
            # P2 selects numbers incrementally until total >= P1's total
            p2_total, selected_numbers, remaining_numbers = incremental_selection(
                remaining_numbers, p2_total, p1_total
            )
            if not selected_numbers:  # If no more valid moves, end the game
                print("No valid moves for P2. Game ends.")
                break
            print(f"P2 chooses: {selected_numbers}, P2 total: {p2_total}")
        
        is_p1_turn = not is_p1_turn  # Switch player turns

        # Break if no more numbers are available to pick
        if not remaining_numbers:
            print("No more numbers remaining. Game ends.")
            break
    
    # Determine and print the winner
    winner = determine_winner(p1_total, p2_total)
    if winner == 1:
        print("\nP1 wins with a total of", p1_total)
    elif winner == -1:
        print("\nP2 wins with a total of", p2_total)
    else:
        print("\nThe game is a tie!")

# Running the game with numbers from 1 to 30
run_expectiminimax_alpha_beta_game(10)



P1's turn
P1 chooses: [6], P1 total: 6

P2's turn
P2 chooses: [7], P2 total: 7

P1's turn
P1 chooses: [10], P1 total: 16

P2's turn
P2 chooses: [4, 8], P2 total: 19

P1's turn
P1 chooses: [9], P1 total: 25

P2's turn
P2 chooses: [2, 1, 5], P2 total: 27

P1's turn
P1 chooses: [3], P1 total: 28
P1 wins the game, no need for further selection

P1 wins with a total of 28


### Catch Up Game - Interactive Mode

#### In this mode Player 1 will Randomly select a number to start the game. Then inser "Next" in the message box for subsequent moves by users. Here subsequent moves are also selected at Random and eqi-priority as mentioned in question.

In [1]:
import random


# Function to determine the winner of the game
def determine_winner(p1_total, p2_total):
    if p1_total > p2_total:
        return 1  # P1 wins
    elif p2_total > p1_total:
        return -1  # P2 wins
    else:
        return 0  # It's a tie


# Function to display the board state (remaining numbers)
def display_board(remaining_numbers, p1_total, p2_total):
    print(f"\nRemaining Numbers: {remaining_numbers}")
    print(f"P1's Total: {p1_total} | P2's Total: {p2_total}")


# Function to simulate random selection of numbers
def random_selection(remaining_numbers, player_total, target_total, player_name):
    selected_numbers = []
    current_total = player_total

    # Keep selecting numbers until the player's total equals or exceeds the opponent's total
    while current_total < target_total and remaining_numbers:
        random_choice = random.choice(remaining_numbers)
        selected_numbers.append(random_choice)
        current_total += random_choice
        remaining_numbers.remove(random_choice)
        print(f"{player_name} selects: {random_choice}")

        if current_total >= target_total:  # Stop when player total meets or exceeds opponent's total
            break

    return current_total, selected_numbers, remaining_numbers


# Main function to run the game interactively using "Next" as input
def run_interactive_game_with_next(n):
    remaining_numbers = list(range(1, n + 1))  # Initialize remaining numbers
    p1_total = 0  # Player 1 total score
    p2_total = 0  # Player 2 total score
    is_p1_turn = True  # Flag to switch between players
    first_turn = True  # Flag for the first move

    while remaining_numbers:
        display_board(remaining_numbers, p1_total, p2_total)

        # Wait for the user to input "Next" to progress
        user_input = input("\nType 'Next' to proceed: ")
        if user_input.lower() != "next":
            print("Invalid input. Please type 'Next' to proceed.")
            continue

        if is_p1_turn:
            print("\nP1's turn")
            if first_turn:  # First move, P1 selects only one number
                print(f"\nFirst move! P1 must choose one number.")
                random_choice = random.choice(remaining_numbers)  # Random first move by P1
                p1_total += random_choice
                remaining_numbers.remove(random_choice)
                selected_numbers = [random_choice]  # Initialize selected_numbers for first move
                print(f"P1 selects: {random_choice}")
                first_turn = False
            else:
                # P1 selects numbers randomly until total >= P2's total
                p1_total, selected_numbers, remaining_numbers = random_selection(
                    remaining_numbers, p1_total, p2_total, "P1"
                )
            print(f"P1's Total: {p1_total}")
            if not selected_numbers:  # If no more valid moves, end the game
                print("No valid moves for P1. Game ends.")
                break
            if p2_total + sum(remaining_numbers) < p1_total:  # If P2 cannot exceed P1
                print("P1 wins the game, no need for further selection")
                break
        else:
            print("\nP2's turn")
            # P2 selects numbers randomly until total >= P1's total
            p2_total, selected_numbers, remaining_numbers = random_selection(
                remaining_numbers, p2_total, p1_total, "P2"
            )
            if not selected_numbers:  # If no more valid moves, end the game
                print("No valid moves for P2. Game ends.")
                break
            print(f"P2's Total: {p2_total}")

        is_p1_turn = not is_p1_turn  # Switch player turns

        # Break if no more numbers are available to pick
        if not remaining_numbers:
            print("No more numbers remaining. Game ends.")
            break

    # Determine and print the winner
    winner = determine_winner(p1_total, p2_total)
    if winner == 1:
        print("\nP1 wins with a total of", p1_total)
    elif winner == -1:
        print("\nP2 wins with a total of", p2_total)
    else:
        print("\nThe game is a tie!")


# Running the interactive game with numbers from 1 to 10
run_interactive_game_with_next(10)



Remaining Numbers: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
P1's Total: 0 | P2's Total: 0
Invalid input. Please type 'Next' to proceed.

Remaining Numbers: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
P1's Total: 0 | P2's Total: 0

P1's turn

First move! P1 must choose one number.
P1 selects: 5
P1's Total: 5

Remaining Numbers: [1, 2, 3, 4, 6, 7, 8, 9, 10]
P1's Total: 5 | P2's Total: 0

P2's turn
P2 selects: 2
P2 selects: 6
P2's Total: 8

Remaining Numbers: [1, 3, 4, 7, 8, 9, 10]
P1's Total: 5 | P2's Total: 8

P1's turn
P1 selects: 4
P1's Total: 9

Remaining Numbers: [1, 3, 7, 8, 9, 10]
P1's Total: 9 | P2's Total: 8

P2's turn
P2 selects: 10
P2's Total: 18

Remaining Numbers: [1, 3, 7, 8, 9]
P1's Total: 9 | P2's Total: 18
Invalid input. Please type 'Next' to proceed.

Remaining Numbers: [1, 3, 7, 8, 9]
P1's Total: 9 | P2's Total: 18

P1's turn
P1 selects: 7
P1 selects: 8
P1's Total: 24

Remaining Numbers: [1, 3, 9]
P1's Total: 24 | P2's Total: 18

P2's turn
P2 selects: 9
P2's Total: 27

Remaining Numbers: 

### Catch Up Game - Interactive Mode - Expecti-MinMax-Alpha-Beta-Pruning

#### In this mode Player 1 will Ranodmly select a number to start the game. Then inser "Next" in the message box for subsequent moves by users. Here subsequent moves are also selected at Random and eqi-priority as mentioned in question.

In [ ]:
import random
import itertools


# Function to determine the winner of the game
def determine_winner(p1_total, p2_total):
    if p1_total > p2_total:
        return 1  # P1 wins
    elif p2_total > p1_total:
        return -1  # P2 wins
    else:
        return 0  # It's a tie


# Function to display the board state (remaining numbers)
def display_board(remaining_numbers, p1_total, p2_total):
    print(f"\nRemaining Numbers: {remaining_numbers}")
    print(f"P1's Total: {p1_total} | P2's Total: {p2_total}")


# Function to calculate a basic heuristic evaluation for game states
def evaluate_game(p1_total, p2_total):
    return p1_total - p2_total


# Expectiminimax algorithm with Alpha-Beta pruning
def expectiminimax_alpha_beta(remaining_numbers, p1_total, p2_total, is_p1_turn, alpha, beta):
    if not remaining_numbers:  # Base case: no numbers left, evaluate the game state
        return evaluate_game(p1_total, p2_total), []

    if is_p1_turn:
        max_eval = float('-inf')
        best_move = []
        for subset in itertools.combinations(remaining_numbers, 1):  # Simulate 1-move picks
            new_remaining_numbers = [x for x in remaining_numbers if x not in subset]
            new_p1_total = p1_total + sum(subset)
            eval, _ = expectiminimax_alpha_beta(new_remaining_numbers, new_p1_total, p2_total, False, alpha, beta)
            if eval > max_eval:
                max_eval = eval
                best_move = subset
            alpha = max(alpha, eval)  # Alpha pruning
            if beta <= alpha:  # Cutoff
                break
        return max_eval, best_move
    else:
        expected_value = 0  # For chance player, compute expected value of all possible outcomes
        for subset in itertools.combinations(remaining_numbers, 1):  # Simulate 1-move picks
            new_remaining_numbers = [x for x in remaining_numbers if x not in subset]
            new_p2_total = p2_total + sum(subset)
            eval, _ = expectiminimax_alpha_beta(new_remaining_numbers, p1_total, new_p2_total, True, alpha, beta)
            expected_value += eval / len(remaining_numbers)  # Compute expected value over all possibilities
        return expected_value, []


# Function for Player 1 to select a number using Expectiminimax
def player_selection_using_expectiminimax(remaining_numbers, p1_total, p2_total):
    print("\nP1 is calculating the best move using Expectiminimax...")
    _, best_move = expectiminimax_alpha_beta(remaining_numbers, p1_total, p2_total, True, float('-inf'), float('inf'))
    return best_move[0] if best_move else None


# Function to simulate random selection of numbers for P2, ensuring it keeps picking until total >= P1's total
def random_selection_until_target(remaining_numbers, player_total, target_total, player_name):
    selected_numbers = []
    current_total = player_total

    # Keep selecting numbers until the player's total equals or exceeds the opponent's total
    while current_total < target_total and remaining_numbers:
        random_choice = random.choice(remaining_numbers)
        selected_numbers.append(random_choice)
        current_total += random_choice
        remaining_numbers.remove(random_choice)
        print(f"{player_name} selects: {random_choice}")

    return current_total, selected_numbers, remaining_numbers


# Main function to run the game interactively using Expectiminimax for P1 and random for P2
def run_expectiminimax_game(n):
    remaining_numbers = list(range(1, n + 1))  # Initialize remaining numbers
    p1_total = 0  # Player 1 total score
    p2_total = 0  # Player 2 total score
    is_p1_turn = True  # Flag to switch between players
    first_turn = True  # Flag for the first move

    while remaining_numbers:
        display_board(remaining_numbers, p1_total, p2_total)

        # Wait for the user to input "Next" to progress
        user_input = input("\nType 'Next' to proceed: ")
        if user_input.lower() != "next":
            print("Invalid input. Please type 'Next' to proceed.")
            continue

        if is_p1_turn:
            print("\nP1's turn")
            if first_turn:  # First move, P1 selects only one number randomly
                print(f"\nFirst move! P1 must choose one number.")
                random_choice = random.choice(remaining_numbers)  # Random first move by P1
                p1_total += random_choice
                remaining_numbers.remove(random_choice)
                print(f"P1 selects: {random_choice}")
                first_turn = False
            else:
                # P1 selects the best move using Expectiminimax
                while p1_total < p2_total:  # Ensure P1 continues until total >= P2
                    best_move = player_selection_using_expectiminimax(remaining_numbers, p1_total, p2_total)
                    if best_move is not None:
                        p1_total += best_move
                        remaining_numbers.remove(best_move)
                        print(f"P1 selects: {best_move}")
            print(f"P1's Total: {p1_total}")
            if not remaining_numbers:  # If no more valid moves, end the game
                break
        else:
            print("\nP2's turn")
            # P2 selects numbers randomly until total >= P1's total
            while p2_total < p1_total:  # Ensure P2 continues until total >= P1
                p2_total, selected_numbers, remaining_numbers = random_selection_until_target(
                    remaining_numbers, p2_total, p1_total, "P2"
                )
                if not selected_numbers:  # If no more valid moves, end the game
                    print("No valid moves for P2. Game ends.")
                    break
            print(f"P2's Total: {p2_total}")

        is_p1_turn = not is_p1_turn  # Switch player turns

        # Break if no more numbers are available to pick
        if not remaining_numbers:
            print("No more numbers remaining. Game ends.")
            break

    # Determine and print the winner
    winner = determine_winner(p1_total, p2_total)
    if winner == 1:
        print("\nP1 wins with a total of", p1_total)
    elif winner == -1:
        print("\nP2 wins with a total of", p2_total)
    else:
        print("\nThe game is a tie!")


# Running the Expectiminimax game with numbers from 1 to 10
run_expectiminimax_game(10)
